<a href="https://colab.research.google.com/github/gha2012/mtsslWizard_colab/blob/main/mtsslWizard_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is a colab running mtsslWizard for bipedal labels.

Enter details for the labelling sites below and press Runtime->Run all...

A "Choose files..." button will appear in the output. Use it to upload a .pdb file.

Once the calculation completes, you can download the ensembles from the file system on the left. The result files are inside the "/content/mtsslWizard_bipedal" folder.

There should be one .pdb file for each ensemble and a .txt file starting with "distr_" that contains the distance distribution.

In [ ]:
#@title Details about the labelling sites: { display-mode: "form" }

labelPositions = []

#first label
#@markdown - Provide details for the first labelling site:
firstLabel = {}
bipedal_1 = True #@param {type:"boolean"}
firstLabel["bipedal"] = bipedal_1

anchor1resi_1 = 6 #@param {type:"integer"}
firstLabel["anchor1resi"] = anchor1resi_1

anchor1chain_1 = "A" #@param {type:"string"}
firstLabel["anchor1chain"] = anchor1chain_1

anchor2resi_1 = 8 #@param {type:"integer"}
firstLabel["anchor2resi"] = anchor2resi_1

anchor2chain_1 = "A" #@param {type:"string"}
firstLabel["anchor2chain"] = anchor2chain_1

label_1 = "nta" #@param ["nta","R1","ida", "rx", "SLIM"]
firstLabel["label"] = label_1

vdWcutoff_1 = 2.0 #@param {type:"number"}
firstLabel["vdWcutoff"] = vdWcutoff_1

clashes_1 = 15 #@param {type:"integer"}
firstLabel["clashes"] = clashes_1

numberOfRuns_1 = 100 #@param {type:"number"}
firstLabel["numberOfRuns"] = numberOfRuns_1

internalClashCheck_1 = True #@param {type: "boolean"}
firstLabel["internalClashCheck"] = internalClashCheck_1

#second label
#@markdown - Provide details for the second labelling site:
secondLabel = {}
bipedal_2 = False #@param {type:"boolean"}
secondLabel["bipedal"] = bipedal_2

anchor1resi_2= 28 #@param {type:"integer"}
secondLabel["anchor1resi"] = anchor1resi_2

anchor1chain_2 = "A" #@param {type:"string"}
secondLabel["anchor1chain"] = anchor1chain_2

anchor2resi_2 = 17 #@param {type:"integer"}
secondLabel["anchor2resi"] = anchor2resi_2

anchor2chain_2 = "A" #@param {type:"string"}
secondLabel["anchor2chain"] = anchor2chain_2

label_2 = "R1" #@param ["nta","R1","ida", "rx", "SLIM"]
secondLabel["label"] = label_2

vdWcutoff_2 =  2.5 #@param {type:"number"}
secondLabel["vdWcutoff"] = vdWcutoff_2

clashes_2 = 6 #@param {type:"integer"}
secondLabel["clashes"] = clashes_2

numberOfRuns_2 = 100 #@param {type:"number"}
firstLabel["numberOfRuns"] = numberOfRuns_2

internalClashCheck_2 = True #@param {type: "boolean"}
firstLabel["internalClashCheck"] = internalClashCheck_2


#first label
labelPositions.append(firstLabel)

#second label
labelPositions.append(secondLabel)

In [ ]:
#@title import modules and libraries ...
%cd /content/
!git clone https://github.com/gha2012/mtsslWizard_bipedal
%cd /content/mtsslWizard_bipedal/
!pip install biopython

In [ ]:
#@title calculate label ensembles...
from mWclasses.distanceDistribution import DistanceDistribution as MWdistanceDistribution
from mWclasses.bipedalEnsemble import BipedalEnsemble
from mWclasses.monopedalEnsemble import MonopedalEnsemble

from Bio.PDB import *
import numpy
import itertools
import os



# get information about labelling sites and labels
from google.colab import files
uploaded = files.upload()
# Get the uploaded file name
file_name = list(uploaded.keys())[0]

# Save the file to the current directory
with open(file_name, 'wb') as f:
    f.write(uploaded[file_name])

# Get the file path
#pdbfile = os.path.abspath(file_name)
pdbfile = file_name


spinPositionsForDistance = []
distributionFilename = ""
for labelPosition in labelPositions:
    thisLabel = None
    if labelPosition["bipedal"]:
        thisLabel = BipedalEnsemble(labelPosition["anchor1resi"], labelPosition["anchor1chain"],
                                    labelPosition["anchor2resi"], labelPosition["anchor2chain"],
                                    labelPosition["label"], pdbfile, labelPosition["vdWcutoff"],
                                    labelPosition["clashes"], labelPosition["numberOfRuns"], labelPosition["maxTries"], labelPosition["internalClashCheck"])
        thisLabel.createRotamers()
        distributionFilename += thisLabel.filename
    else:
        thisLabel = MonopedalEnsemble(labelPosition["anchor1resi"], labelPosition["anchor1chain"],
                                    labelPosition["label"], pdbfile, labelPosition["vdWcutoff"],
                                    labelPosition["clashes"])
        thisLabel.createRotamers()
        distributionFilename += thisLabel.filename
    spinPositionsForDistance.append(thisLabel.spinPositions)
    distributionFilename += "_"

dist = []
for pair in itertools.combinations(spinPositionsForDistance, 2):
    # print pair
    mwDistanceDistribution = MWdistanceDistribution()
    pair_distances = mwDistanceDistribution.calculateDistanceDistribution(pair[0], pair[1])
    # print pair_distances
    dist.extend(pair_distances)
histogram = numpy.histogram(dist, numpy.arange(100))
envelopePlot = numpy.zeros((100, 2))
envelopePlot[0:99] = numpy.column_stack((histogram[1][0:len(histogram[1]) - 1], histogram[0]))

# put point in mid of bin
envelopePlot[:, 0] += 0.5
normEnvelopePlot = numpy.copy(envelopePlot)
normEnvelopePlot[:, 1] = normEnvelopePlot[:, 1] / numpy.amax(histogram[0])

# combine dist and histogram to single array before output
output = numpy.column_stack((envelopePlot, normEnvelopePlot[:, 1]))
averageDistance = numpy.average(dist)
# print averageDistance
distributionString = "["
for row in output:
    # print row
    x = row[0]
    y = row[2]
    # print x, y
    newPoint = "{x:%1.2f, y:%1.2f}," % (x, y)
    distributionString += newPoint
distributionString += "]"
csvString = "%s\n" % averageDistance
for row in output:
    # print row
    x = row[0]
    y = row[2]
    # print x, y
    newPoint = "%1.2f\t%1.2f\n" % (x, y)
    csvString += newPoint
distributionFilename = distributionFilename[:-1] # remove trailing _
distributionFilename += ".txt"
distributionFilename = distributionFilename.replace(pdbfile, "") #remove repeated pdbfilename
distributionFilename = "distr_" + pdbfile + "_" + distributionFilename #add pdbfilename once
with open(distributionFilename,"w") as text_file:
    text_file.write(csvString)